# Transition Metal Ions with $d^1$ & $d^2$ Electron Configurations



In [1]:
%load_ext autoreload
# provide cell timings
%load_ext autotime
# %autoreload 2
import sympy as sp
import numpy as np
import math
from scipy.special import sph_harm as Ynm_num
# from sympy import poly, lambdify
from IPython.display import display, Math
# %config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from qdef import *
from misc import *
from itertools import product
from sympy import I
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import HTML, display, Math, Latex
from matplotlib import pyplot as plt
%matplotlib widget
from pyperclip import copy as pypercopy
from collections import OrderedDict

Reloading /Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


In [2]:
CFparams = pd.read_pickle(os.path.join(module_dir,'data','CFparams.pkl'))
crystal_splits = pickle.load(open(os.path.join(module_dir,'data','crystal_splits.pkl'),'rb'))

## $d^2$ (Nov 11)

The general goal of this section is to be able to programatically reproduce Ch. 2 of TSK for other groups other than cubic.

Then to apply this analysis to as many specific systems for which CF params are known.

Ideally I should also know at this stage how is that two valence holes can be made equivalent to two valence electrons so that the d^2 electron configurations are made equivalent to d^8 configurations (such as that of Ni^2+).

The general goal can be split into the following tasks:

- Calculation of slater determinant symbols together with visualization.

- Calculation of term symbols.

- Printout of reduction tables (i.e. Table 2.1 for other groups).

- Printout of CG coefficients for the refined Vee coefficients.

- Generation of total spin wavefunctions for two electrons.

- Number of predicted states according to Pauli exclusion principle.

- Calculation of allowed terms.

- Calculation of terms and associated wavefunctions (i.e. analogues of Table 2.2, 2.3, 2.4, ...).

- Calculation of term energies, in terms of slater integrals or racah parameters.

- Visualization of energy levels for different two-electron configurations (diagrams such as that in Fig 2.3).

- Visualization of some sort of the spatial dependence of these eigenfunctions.

- Two electron integrals in terms of Racah parameters.

- Two-Electron wavefunctions.

### Improving notation for components

In [640]:
# def notation_maker(group)
greek = 'alpha beta gamma zeta eta mu nu xi phi chi omega'
greek_alphabet = [sp.Symbol(r'\%s' % g) for g in greek.split(' ')]
new_labels = []
for group_label in CPGs.all_group_labels:
    group = CPGs.get_group_by_label(group_label)
    # get the irrep symbols
    irrep_symbols = group.irrep_labels
    # get the irrep dims
    irrep_dims = group.irrep_dims
    # separate them according to their dimension
    dims = list(set(irrep_dims.values()))
    irrep_symbols = OrderedDict([(dim, ([isymb for isymb in irrep_symbols if irrep_dims[isymb]==dim])) for dim in dims])
    irrep_symbols = OrderedDict([(k,sorted(v, key = lambda x: sp.latex(x))) for k,v in irrep_symbols.items()])
    irrep_symbols = sum(list(irrep_symbols.values()),[])
    asymb = str(irrep_symbols[0])
    if 'g' in asymb or 'u' in asymb:
        irrep_symbols_u = [i for i in irrep_symbols if 'u' in str(i)]
        irrep_symbols_v = [i for i in irrep_symbols if 'g' in str(i)]
        component_labels = {ir:OrderedDict() for ir in group.irrep_labels}
        counter = 0
        for irrep_symbol in irrep_symbols_u:
            old_components = group.component_labels[irrep_symbol]
            for old_component in old_components:
                component_labels[irrep_symbol][old_component] = sp.Symbol('{%s}_{u}' % (sp.latex(greek_alphabet[counter])))
                counter += 1
        counter = 0
        for irrep_symbol in irrep_symbols_v:
            old_components = group.component_labels[irrep_symbol]
            for old_component in old_components:
                component_labels[irrep_symbol][old_component] = sp.Symbol('{%s}_{g}' % (sp.latex(greek_alphabet[counter])))
                counter += 1
    elif group_label in ['C_{6h}','C_{4h}']:
        component_labels = {ir:OrderedDict() for ir in group.irrep_labels}
        counter = 0
        for irrep_symbol in irrep_symbols:
            old_components = group.component_labels[irrep_symbol]
            for old_component in old_components:
                component_labels[irrep_symbol][old_component] = sp.Symbol('{\gamma}_{%d}' % (counter+1))
                counter += 1
    else:
        component_labels = {ir:OrderedDict() for ir in group.irrep_labels}
        counter = 0
        for irrep_symbol in irrep_symbols:
            old_components = group.component_labels[irrep_symbol]
            for old_component in old_components:
                component_labels[irrep_symbol][old_component] = sp.Symbol('{%s}' % sp.latex(greek_alphabet[counter]))
                counter += 1
    new_labels.append((group_label, component_labels))
new_labels = OrderedDict(new_labels)

In [169]:
outlines = []
for group_label in CPGs.all_group_labels:
    outlines.append("\\subsection{Group : ${%s}$}" % group_label)
#     if group_label != 'O':
#         continue
    group = CPGs.get_group_by_label(group_label)
    irrep_symbols = group.irrep_labels
    components = group.component_labels
    for irrep in irrep_symbols:
        these_components = ','.join(list(map(sp.latex,components[irrep])))
        these_new_components = ','.join(list(map(lambda x: sp.latex(new_labels[group_label][irrep][x]),components[irrep])))
        outlines.append('\n\\noindent$%s: %s $\n\\vspace{0.1cm}\n' % (sp.latex(irrep),these_components))
        outlines.append('\n\\noindent$%s: %s $\n\\vspace{0.5cm}\n' % (sp.latex(irrep),these_new_components))
final_output = '\n'.join(outlines).replace("^'","^{'}").replace("^''","^{''}")
open('/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/ComponentLabeling-rosetta.tex','w').write(final_output)

18473

### Spin-orbitals (see qdef-tables.tex)

In [3]:
new_labels = pickle.load(open('./Data/components_rosetta.pkl','rb'))

In [4]:
# with new labels
group_chunks = []
for group_label in CPGs.all_group_labels:
    group = CPGs.get_group_by_label(group_label)
    all_spin_combos = OrderedDict()
    spin_states = [sp.Symbol(r'\alpha'), sp.Symbol(r'\beta')]
    group.new_component_labels = {ir:list(new_labels[group_label][ir].values()) for ir in group.irrep_labels}
    for irrep in group.irrep_labels:
        spin_orbitals = []
        for component0, component1 in product(group.new_component_labels[irrep],group.new_component_labels[irrep]):
            for s0, s1 in product(spin_states,spin_states):
                spin_orbital = (component0, component1, s0, s1)
                if (component1, component0, s1, s0) not in spin_orbitals:
                    if (component1, s1) != (component0, s0):
                        spin_orbitals.append(spin_orbital)
        spin_orbital_symbols = {}
        for spin_orbital in spin_orbitals:
            component0, component1, s0, s1 = spin_orbital
            if s0 == spin_states[0]:
                cstar0 = sp.latex(component0)
            else:
                cstar0 = r'\overline{%s}' % sp.latex(component0)
            if s1 == spin_states[0]:
                cstar1 = sp.latex(component1)
            else:
                cstar1 = r'\overline{%s}' % sp.latex(component1)
            spin_orbital_symbols[spin_orbital] = sp.Symbol(r'$|%s%s|$' % (cstar0, cstar1))
        all_slater_symbols = []
        for s in list(spin_orbital_symbols.values()):
            all_slater_symbols.append(sp.latex(s))
        all_slater_symbols = ', '.join(all_slater_symbols)
        all_slater_symbols = '\\begin{center}\n%s\n\\end{center}' % all_slater_symbols
        all_slater_symbols = ('\\begin{center}\n $%s{\\cdot}%s:$ \n\\end{center}\n\n' % (sp.latex(irrep),sp.latex(irrep))) + all_slater_symbols
        all_spin_combos[irrep] = all_slater_symbols
    final_output = '\n\n'.join(all_spin_combos.values())
    final_output = ('\\hrulefill \n \\begin{center} Group $%s$ \\end{center} \n' % group_label) + final_output
    group_chunks.append(final_output)
super_output = '\n'.join(group_chunks)
super_output = super_output.replace("^'","^{'}").replace("^''","^{''}")
open('//Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/spin-orbitals.tex','w').write(super_output)

31183

In [638]:
pickle.dump(new_labels, open('./Data/components_rosetta.pkl','wb'))

In [639]:
# group_chunks = []
# for group_label in CPGs.all_group_labels:
#     group = CPGs.get_group_by_label(group_label)
#     all_spin_combos = OrderedDict()
#     spin_states = [sp.Symbol(r'\alpha'), sp.Symbol(r'\beta')]
#     for irrep in group.irrep_labels:
#         spin_orbitals = []
#         for component0, component1 in product(group.component_labels[irrep],group.component_labels[irrep]):
#             for s0, s1 in product(spin_states,spin_states):
#                 spin_orbital = (component0, component1, s0, s1)
#                 if (component1, component0, s1, s0) not in spin_orbitals:
#                     if (component1, s1) != (component0, s0):
#                         spin_orbitals.append(spin_orbital)
#         spin_orbital_symbols = {}
#         for spin_orbital in spin_orbitals:
#             component0, component1, s0, s1 = spin_orbital
#             if s0 == spin_states[0]:
#                 cstar0 = sp.latex(component0)
#             else:
#                 cstar0 = r'\overline{%s}' % sp.latex(component0)
#             if s1 == spin_states[0]:
#                 cstar1 = sp.latex(component1)
#             else:
#                 cstar1 = r'\overline{%s}' % sp.latex(component1)
#             spin_orbital_symbols[spin_orbital] = sp.Symbol(r'$|%s%s|$' % (cstar0, cstar1))
#         all_slater_symbols = []
#         for s in list(spin_orbital_symbols.values()):
#             all_slater_symbols.append(sp.latex(s))
#         all_slater_symbols = ', '.join(all_slater_symbols)
#         all_slater_symbols = '\\begin{center}\n%s\n\\end{center}' % all_slater_symbols
#         all_slater_symbols = ('\\begin{center}\n $%s{\\cdot}%s:$ \n\\end{center}\n\n' % (sp.latex(irrep),sp.latex(irrep))) + all_slater_symbols
#         all_spin_combos[irrep] = all_slater_symbols
#     final_output = '\n\n'.join(all_spin_combos.values())
#     final_output = ('\\hrulefill \n \\begin{center} Group $%s$ \\end{center} \n' % group_label) + final_output
#     group_chunks.append(final_output)
# super_output = '\n'.join(group_chunks)
# super_output = super_output.replace("^'","^{'}").replace("^''","^{''}")
# open('//Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/spin-orbitals.tex','w').write(super_output)

### Terms and their wavefunctions

In [5]:
from sympy.physics.wigner import clebsch_gordan as ClebschG

In [6]:
def det_simplify(qet):
    '''
    Simplification from antisymmetry of composing elements.
    '''
    qet_dict = qet.dict
    best_qet = {}
    for key, coeff in qet_dict.items():
        ikey = (*key[3:],*key[:3])
        current_keys = list(best_qet.keys())
        if ikey in current_keys:
            best_qet[ikey] += -coeff
            continue
        if key not in current_keys:
            best_qet[key] = coeff
        else:
            best_qet[key] += coeff
    return Qet(best_qet)

In [8]:
# this is now included in qdefcore
# class Term():
#     '''
#     To  represent  a  term,  this  object  holds the states that
#     correspond  to  it,  and offers a few methods in to view the
#     enclosed wave functions.
#     '''
#     def __init__(self, init_dict):
#         for k,v in init_dict.items():
#             setattr(self, k, v)
#         self.term_prototype = sp.Symbol(r'{{}}^{{{M}}}{ir}')
#         self.state_label_prototype = sp.Symbol(r'\Psi({α}, {S}, {{{Γ}}}, {M_s}, {γ})')
#     def term_symbol(self):
#         return sp.Symbol(str(self.term_prototype).format(M=str(2*self.S+1),ir=str(sp.latex(self.irrep))))
#     def make_state_symbols(self):
#         state_symbols = []
#         for qet, state_key in zip(self.states, self.state_keys):
#             (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
#             ket = qet.as_ket(fold_keys=True)
#             ket_symbol = sp.latex(ket).replace('\\right\\rangle','\\right|')
#             α = Γ1*Γ2
#             term_symb = self.term_symbol()
#             state_symbol = '\\Psi(%s,%s,M\!=\!%d,%s)' % (sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
#             state_symbols.append((state_symbol, ket_symbol))
#         return state_symbols
#     def __str__(self):
#         return (self.term_symbol())
#     def __repr__(self):
#         return '%s: %d states' % (str(self.term_symbol()), len(self.states))

In [9]:
def as_determinantal_ket(qet):
    qdict = {}
    for k,v in qet.dict.items():
        if k[1] >= 0:
            k0 = sp.Symbol(str(k[0]))
        else:
            k0 = sp.Symbol('\\overline{%s}'%str(k[0]))
        if k[3] >= 0:
            k1 = sp.Symbol(str(k[2]))
        else:
            k1 = sp.Symbol('\\overline{%s}'%str(k[2]))
        qdict[(k0,k1)] = v
    qet = Qet(qdict)
    qet = qet*(1/qet.norm())
    ket = qet.as_ket(fold_keys=True)
    ket = sp.latex(ket).replace('\\right\\rangle','\\right|')
    return sp.Symbol(ket), qet

In [11]:
# with new labels and checking for num of waves
def num_waves(ir0,ir1):
    if ir0 == ir1:
        return sp.binomial(group.irrep_dims[ir0]*2, 2)
    else:
        return group.irrep_dims[ir0]*group.irrep_dims[ir1]*4
def full_waves():
    done = []
    size = 0
    for k,v in group.product_table.odict.items():
        if (k[1],k[0]) in done:
            continue
        if k[1] == k[0]:
            size += sp.binomial(group.irrep_dims[k[1]]*2,2)
        else:
            size += sum(list(map(lambda x: group.irrep_dims[x]*4, v)))
        done.append(k)
    return size
all_final_outputs = []
tally_waves = {}
single_col_groups = ['C_{1}','C_{2}']
num_multicols = {group_label:2 for group_label in CPGs.all_group_labels}
for scg in single_col_groups:
    num_multicols[scg] = 1
archival_terms = {}
for group_counter, group_label in enumerate(CPGs.all_group_labels):
#     if group_label != 'O':
#         continue
    print("Working on group %s ..." % group_label)
    group = CPGs.get_group_by_label(group_label)
    print("computing partial sums...")
    s1, s2 = sp.S(1)/2, sp.S(1)/2
    Ss  = [0,1]
    m1s = [-sp.S(1)/2, sp.S(1)/2]
    m2s = [-sp.S(1)/2, sp.S(1)/2]
    group_CGs = group.CG_coefficients
    flat_labels = dict(sum([list(l.items()) for l in list(new_labels[group_label].values())],[]))
    group_CGs = {(flat_labels[k[0]], flat_labels[k[1]], flat_labels[k[2]]):v for k,v in group_CGs.items()}
    summands = OrderedDict()
    group.new_component_labels = OrderedDict([(ir, list(new_labels[group_label][ir].values())) for ir in group.irrep_labels])
    for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
                                     group.irrep_labels,
                                     group.irrep_labels,
                                     m1s,
                                     m2s,
                                     Ss):
        for γ1, γ2, γ3 in product(group.new_component_labels[Γ1],
                                  group.new_component_labels[Γ2],
                                  group.new_component_labels[Γ3]):
            for mSz in range(S,-S-1,-1):
                sCG = ClebschG(s1, s2, S, m1, m2, mSz)
                if (γ1, γ2, γ3) not in group_CGs.keys():
                    continue
                else:
                    gCG = group_CGs[(γ1, γ2, γ3)]
                coeff = sCG*gCG
                key = (Γ1, Γ2, Γ3, γ3, S, mSz)
                if (Γ2,m2,γ2) == (Γ1,m1,γ1):
                    continue
                if coeff!=0:
                    if key not in summands.keys():
                        summands[key] = []
                    summands[key].append(Qet({(Γ1,γ1,m1,Γ2,γ2,m2):coeff}))
    print("summing out inner parts ...")
    total_qets = OrderedDict([(k, sum(v,Qet({}))) for k,v in summands.items()])
    best_qets = OrderedDict([(k, det_simplify(v)) for k,v in total_qets.items()])
    best_qets = OrderedDict([(k, v) for k,v in best_qets.items() if len(v.dict)!=0])
    print("term collection truck ...")
    terms = OrderedDict()
    done_keys = []
    for k, v in best_qets.items():
        (Γ1, Γ2, Γ3, γ3, S, mSz) = k
        equiv_key = (Γ2, Γ1, Γ3, γ3, S, mSz)
        term_pair = (Γ3, S)
        if term_pair not in terms.keys():
            terms[term_pair] = OrderedDict()
        if equiv_key in done_keys:
            continue
        terms[term_pair][k] = v
        done_keys.append(k)
    final_terms = OrderedDict()
    for term in terms:
        ir, S = term
        states = terms[term]
        one_term = Term({'irrep': ir, 'S': S, 'states': states})
        final_terms[term] = one_term
    print("Manufacturing latex output ....")
    max_counter = 3
    if group_counter == 0:
        print_outs = ['\n\\newpage\n\n\\section{Terms and wave functions}\n\\subsection{Group $%s$}\n\n\\begin{center} \\underline{Component labels} \n\\vspace{0.2cm}\n' % group_label]
    else:
        print_outs = ['\n\\doublerulefill\n\\subsection{Group $%s$} \n\n\\begin{center}\n\n\\underline{Component labels} \n\\vspace{0.2cm}\n' % group_label]
    for running_idx, irrep_symbol in enumerate(group.irrep_labels):
        components_for_printing = ','.join(list(map(sp.latex, group.new_component_labels[irrep_symbol])))
        if running_idx < (len(group.irrep_labels)-1):
            print_outs.append('$%s:\\{%s\\}$ || ' % (sp.latex(irrep_symbol), components_for_printing))
        else:
            print_outs.append('$%s:\\{%s\\}$' % (sp.latex(irrep_symbol), components_for_printing))
    if num_multicols[group_label] == 1:
        print_outs.append('\\end{center}\n\n')
    else:
        print_outs.append('\\end{center}\n\n\\begin{multicols}{%d}\n\n' % num_multicols[group_label])
    supreme_states = {}
    total_waves = 0
    end_terms = {}
    for one_term_k, one_term in final_terms.items():
        term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*one_term_k[1]+1), ir = sp.latex(one_term_k[0]))
        term_symb = sp.Symbol(term_symb)
        print_outs.append('\n\\hrulefill\n\\subsubsection{$%s$}\n\\vspace{0.25cm}\n \\begin{center} \n' % sp.latex(term_symb))
        counter = 0
        prev_α = ''
        end_terms[one_term_k] = []
        origins = []
        for state_key, state in one_term.states.items():
            (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
            α = Γ1 * Γ2
            multiplicity = 2*S+1
            term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*S+1), ir = sp.latex(Γ3))
            state_symbol = '\\Psi_{%d}(%s,%s,M\!=\!%d,%s)' % (counter+1, sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
            state_symbol = sp.Symbol(state_symbol)
            v_simple = Qet({(k[1],k[2],k[4],k[5]):v for k,v in state.dict.items()})
            v_det, q_qet = as_determinantal_ket(v_simple)
            end_terms[one_term_k].append(q_qet)
            origins.append(state_key)
            sup_key = α
            if sup_key not in supreme_states.keys():
                supreme_states[sup_key] = []
            supreme_states[sup_key].append(v_det)
            pout = "$\\textcolor{blue}{%s} = %s$\\vspace{0.1cm}\n" % (sp.latex(state_symbol), sp.latex(v_det))
            pout = sp.Symbol(pout)
            if prev_α != α:
                if prev_α == '':
                    print_outs.append('\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
                else:
                    print_outs.append('\n\\vspace{0.25cm}\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
            prev_α = α
            print_outs.append(sp.latex(pout))
            counter += 1
            total_waves += 1
            done_keys.append(state_key)
        end_terms[one_term_k] = Term({'irrep': one_term_k[0], 'S': one_term_k[1], 'states': end_terms[one_term_k], 'state_keys': origins})
        print_outs.append('\n\\end{center}\n')
    archival_terms[group_label] = end_terms
    if num_multicols[group_label] != 1:
        print_outs.append('\n\\end{multicols}\n')
    irrep_combos = list(combinations_with_replacement(group.irrep_labels,2))
    total_waves_groundtruth = sum([num_waves(ir0,ir1) for ir0, ir1 in irrep_combos])
    tally_waves[group_label] = (total_waves,total_waves_groundtruth,full_waves())
    all_wavefunctions = '\n'.join(print_outs)
    all_wavefunctions = all_wavefunctions.replace("^'","^{'}").replace("^''","^{''}")
    all_final_outputs.append(all_wavefunctions)
print("saving to file ...")
super_final_output = '\n'.join(all_final_outputs)
open('/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/termwaves.tex','w').write(super_final_output)
!beep
print("Saving to pickle ...")
pickle.dump(archival_terms, open('./Data/2e-terms.pkl','wb'))

Working on group C_{1} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{i} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{2} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{s} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{2h} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group D_{2} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{2v} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group D_{2h} .

In [537]:
for k,v in tally_waves.items():
    print(k,v)

C_{1} (1, 1, 1)
C_{i} (6, 6, 6)
C_{2} (6, 6, 6)
C_{s} (6, 6, 6)
C_{2h} (28, 28, 28)
D_{2} (28, 28, 28)
C_{2v} (28, 28, 28)
D_{2h} (120, 120, 120)
C_{4} (28, 28, 28)
S_{4} (28, 28, 28)
C_{4h} (120, 120, 120)
D_{4} (66, 66, 66)
C_{4v} (66, 66, 66)
D_{2d} (66, 66, 66)
D_{4h} (276, 276, 276)
C_{3} (15, 15, 15)
S_{6} (66, 66, 66)
D_{3} (28, 28, 28)
C_{3v} (28, 28, 28)
D_{3d} (120, 120, 120)
C_{6} (66, 66, 66)
C_{3h} (66, 66, 66)
C_{6h} (276, 276, 276)
D_{6} (120, 120, 120)
C_{6v} (120, 120, 120)
D_{3h} (120, 120, 120)
D_{6h} (496, 496, 496)
T (66, 66, 66)
T_{h} (264, 276, 264)
O (190, 190, 190)
T_{d} (190, 190, 190)
O_{h} (780, 780, 780)


In [489]:
group_label = 'T_{h}'
group = CPGs.get_group_by_label(group_label)
irreps = group.irrep_labels
irrep_combos = list(combinations_with_replacement(group.irrep_labels,2))
total_waves_groundtruth = sum([num_waves(ir0,ir1) for ir0, ir1 in irrep_combos])

In [490]:
total_waves_asfound = sum([len(v) for k,v in supreme_states.items()])
total_waves_asfound

264

In [4]:
# class Term():
#     '''
#     Represents a term, holds the states that correspond to it,
#     and offers a display of the term symbol and enclosed wavefunctions.
#     '''
#     def __init__(self, init_dict):
#         for k,v in init_dict.items():
#             setattr(self, k, v)
#         self.term_prototype = sp.Symbol(r'{{}}^{{{M}}}{ir}')
#         self.state_prototype = sp.Symbol(r'\Psi({α}, {S}, {{{Γ}}}, {M_s}, {γ})')
#     def term_symbol(self):
#         return sp.Symbol(str(self.term_prototype).format(M=str(2*self.S+1),ir=str(sp.latex(self.irrep))))
#     def make_state_symbols(self):
#         state_symbols = []
#         for state in self.states:
#             ndict = {
#             'α' : state['qnums'][sp.Symbol(r'\alpha')],
#             'S' : self.S,
#             'Γ' : self.irrep,
#             'M_s' : state['qnums'][sp.Symbol(r'M_s')],
#             'γ' : state['qnums'][sp.Symbol(r'\gamma')]
#             }
#             state_symbols.append(sp.Symbol(str(self.state_prototype).format(**ndict)))
#         return state_symbols
#     def __str__(self):
#         return (self.term_symbol())
#     def __repr__(self):
#         return str(self.term_symbol())

In [5]:
# all_terms = []
# group_label = 'O'
# group = CPGs.get_group_by_label(group_label)
# for irrep in group.irrep_labels:
#     for S in [0,1]:
#         for mS in range(-S, S+1):
#             states = [{'qnums':{sp.Symbol(r'\alpha'):sp.Symbol(r'\alpha'), # qnum that distinguishes between states
#                                 sp.Symbol(r'\gamma'): component, # symbol for a component of a basis for the irrep
#                                 sp.Symbol(r'M_s'):mS # the Ms equal to m_1 + m_2
#                                 }} for component in group.component_labels[irrep]]
#             aterm = Term({'states':states,
#                   'group_label': group_label,
#                   'S': S,
#                   'irrep': irrep
#                  })
#             all_terms.append(aterm)

In [6]:
# for term in all_terms:
#     display(term.term_symbol())
#     for state_symb in term.make_state_symbols():
#         display(state_symb)
#     break

In [7]:
# s1, s2 = sp.S(1)/2, sp.S(1)/2
# Ss = [0,1]
# m1s = [-sp.S(1)/2,sp.S(1)/2]
# m2s = [-sp.S(1)/2,sp.S(1)/2]
# group_CGs = group.CG_coefficients
# Γ = sp.Symbol('T_1')
# summands = {}
# for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
#                                  group.irrep_labels,
#                                  group.irrep_labels,
#                                  m1s, m2s, Ss):
#     for γ1, γ2, γ3 in product(group.component_labels[Γ1],
#                               group.component_labels[Γ2],
#                               group.component_labels[Γ3]):
#         for mSz in range(-S, S+1):
#             sCG = ClebschG(s1, s2, S, m1, m2, mSz)
#             if (γ1, γ2, γ3) not in group_CGs.keys():
#                 gCG = 0
#                 continue
#             else:
#                 gCG = group_CGs[(γ1, γ2, γ3)]
#             coeff = sCG*gCG
#             key = (Γ1, Γ2, Γ3, γ3, mSz)
#             if (Γ2,m2,γ2) == (Γ1,m1,γ1):
#                 coeff = 0
#             if coeff!=0:
#                 if key not in summands.keys():
#                     summands[key] = {}
#                 qkey = (Γ1,m1,γ1,Γ2,m2,γ2)
#                 ikey = (Γ2,m2,γ2Γ1,m1,γ1)
#                 if ikey in summands.keys():
#                     summands[ikey].append(Qet({:-coeff}))
#                 else:
#                     summands[key].append(Qet({(Γ1,m1,γ1,Γ2,m2,γ2):coeff}))

In [603]:
# this just does the work for a single group
# print("computing partial sums...")
# s1, s2 = sp.S(1)/2, sp.S(1)/2
# Ss = [0,1]
# m1s = [-sp.S(1)/2,sp.S(1)/2]
# m2s = [-sp.S(1)/2,sp.S(1)/2]
# group_CGs = group.CG_coefficients
# Γ = sp.Symbol('T_1')
# summands = {}
# for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
#                                  group.irrep_labels,
#                                  group.irrep_labels,
#                                  m1s, m2s, Ss):
#     for γ1, γ2, γ3 in product(group.component_labels[Γ1],
#                               group.component_labels[Γ2],
#                               group.component_labels[Γ3]):
#         for mSz in range(-S, S+1):
#             sCG = ClebschG(s1, s2, S, m1, m2, mSz)
#             if (γ1, γ2, γ3) not in group_CGs.keys():
#                 gCG = 0
#                 continue
#             else:
#                 gCG = group_CGs[(γ1, γ2, γ3)]
#             coeff = sCG*gCG
#             key = (Γ1, Γ2, Γ3, γ3, S, mSz)
#             if (Γ2,m2,γ2) == (Γ1,m1,γ1):
#                 coeff = 0
#             if coeff!=0:
#                 if key not in summands.keys():
#                     summands[key] = []
#                 summands[key].append(Qet({(Γ1,γ1,m1,Γ2,γ2,m2):coeff}))
# print("summing out inner parts ...")
# total_qets = {k:sum(v,Qet({})) for k,v in summands.items()}
# best_qets = {k:det_simplify(v) for k,v in total_qets.items()}
# best_qets = {k:v for k,v in best_qets.items() if len(v.dict)!=0}
# print("term collection truck ...")
# terms = {}
# for k, v in best_qets.items():
#     (Γ1, Γ2, Γ3, γ3, S, mSz) = k
#     term_pair = (Γ3, S)
#     if term_pair not in terms.keys():
#         terms[term_pair] = {}
#     terms[term_pair][k] = v
# final_terms = {}
# for term in terms:
#     ir, S = term
#     states = terms[term]
#     one_term = Term({'irrep': ir, 'S': S, 'states': states})
#     final_terms[term] = one_term
# print("Manufacturing latex output ....")
# max_counter = 3
# print_outs = []
# for one_term_k, one_term in final_terms.items():
#     term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*one_term_k[1]+1), ir = sp.latex(one_term_k[0]))
#     term_symb = sp.Symbol(term_symb)
#     print_outs.append('\n\\hrulefill\n \\begin{center} {\\Large $%s$} \n\\vspace{0.5cm}\n' % sp.latex(term_symb))
#     counter = 0
#     for state_key, state in one_term.states.items():
#         (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
#         α = Γ1 * Γ2
#         term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*S+1), ir = sp.latex(Γ3))
#         state_symbol = '\\Psi(%s,%s,M\!=\!%d,%s)' % (sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
#         state_symbol = sp.Symbol(state_symbol)
#         v_simple = Qet({(k[1],k[2],k[4],k[5]):v for k,v in state.dict.items()})
#         v_det = as_determinantal_ket(v_simple)
#         pout = "$%s = %s$\n" % (sp.latex(state_symbol), sp.latex(v_det))
#         pout = sp.Symbol(pout)
#         print_outs.append(sp.latex(pout))
#         if counter % 5 == 4:
#             print_outs.append('\n\\vspace{0.5cm}\n')
#         counter += 1
#     print_outs.append('\n\\end{center}')
# all_wavefunctions = '\n'.join(print_outs)
# print("saving to file ...")
# open('/Volumes/GoogleDrive/My Drive/Zia Lab/Theoretical Division/termwaves.tex','w').write(all_wavefunctions)

In [445]:
# # with new labels
# def num_waves(ir0,ir1):
#     if ir0 == ir1:
#         return sp.binomial(group.irrep_dims[ir0]+group.irrep_dims[ir1], 2)
#     else:
#         return group.irrep_dims[ir0]*group.irrep_dims[ir1]*4
# def full_waves():
#     done = []
#     size = 0
#     for k,v in group.product_table.odict.items():
#         if (k[1],k[0]) in done:
#             continue
#         size += sum(list(map(lambda x: group.irrep_dims[x], v)))
#         done.append(k)
#     return 2*size
# all_final_outputs = []
# tally_waves = {}
# for group_counter, group_label in enumerate(CPGs.all_group_labels):
#     if group_label != 'T_{h}':
#         continue
#     print("Working on group %s ..." % group_label)
#     group = CPGs.get_group_by_label(group_label)
#     print("computing partial sums...")
#     s1, s2 = sp.S(1)/2, sp.S(1)/2
#     Ss  = [0,1]
#     m1s = [-sp.S(1)/2, sp.S(1)/2]
#     m2s = [-sp.S(1)/2, sp.S(1)/2]
#     group_CGs = group.CG_coefficients
#     flat_labels = dict(sum([list(l.items()) for l in list(new_labels[group_label].values())],[]))
#     group_CGs = {(flat_labels[k[0]], flat_labels[k[1]], flat_labels[k[2]]):v for k,v in group_CGs.items()}
#     summands = OrderedDict()
#     group.new_component_labels = OrderedDict([(ir, list(new_labels[group_label][ir].values())) for ir in group.irrep_labels])
#     for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
#                                      group.irrep_labels,
#                                      group.irrep_labels,
#                                      m1s,
#                                      m2s,
#                                      Ss):
#         for γ1, γ2, γ3 in product(group.new_component_labels[Γ1],
#                                   group.new_component_labels[Γ2],
#                                   group.new_component_labels[Γ3]):
#             for mSz in range(-S, S+1):
#                 sCG = ClebschG(s1, s2, S, m1, m2, mSz)
#                 if (γ1, γ2, γ3) not in group_CGs.keys():
#                     continue
#                 else:
#                     gCG = group_CGs[(γ1, γ2, γ3)]
#                 coeff = sCG*gCG
#                 key = (Γ1, Γ2, Γ3, γ3, S, mSz)
#                 if (Γ2,m2,γ2) == (Γ1,m1,γ1):
#                     continue
#                 if coeff!=0:
#                     if key not in summands.keys():
#                         summands[key] = []
#                     summands[key].append(Qet({(Γ1,γ1,m1,Γ2,γ2,m2):coeff}))
#     print("summing out inner parts ...")
#     total_qets = OrderedDict([(k, sum(v,Qet({}))) for k,v in summands.items()])
#     best_qets = OrderedDict([(k, det_simplify(v)) for k,v in total_qets.items()])
#     best_qets = OrderedDict([(k, v) for k,v in best_qets.items() if len(v.dict)!=0])
#     print("term collection truck ...")
#     terms = OrderedDict()
#     for k, v in best_qets.items():
#         (Γ1, Γ2, Γ3, γ3, S, mSz) = k
#         term_pair = (Γ3, S)
#         if term_pair not in terms.keys():
#             terms[term_pair] = OrderedDict()
#         terms[term_pair][k] = v
#     final_terms = OrderedDict()
#     for term in terms:
#         ir, S = term
#         states = terms[term]
#         one_term = Term({'irrep': ir, 'S': S, 'states': states})
#         final_terms[term] = one_term
#     print("Manufacturing latex output ....")
#     max_counter = 3
#     print_outs = ['\\subsection{Group $%s$} \n \\begin{center} \\underline{Component labels} \n\\vspace{0.2cm}\n' % group_label]
#     for irrep_symbol in group.irrep_labels:
#         components_for_printing = ','.join(list(map(sp.latex, group.new_component_labels[irrep_symbol])))
#         print_outs.append('\\noindent$%s:\\{%s\\}$\n\n' % (sp.latex(irrep_symbol), components_for_printing))
#     print_outs.append('\\end{center}')
#     supreme_states = {}
#     total_waves = 0
#     for one_term_k, one_term in final_terms.items():
#         term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*one_term_k[1]+1), ir = sp.latex(one_term_k[0]))
#         term_symb = sp.Symbol(term_symb)
#         print_outs.append('\n\\hrulefill\n  \\subsubsection{$%s$} \n\\vspace{0.25cm}\n \\begin{center} \n' % sp.latex(term_symb))
#         counter = 0
#         done_keys = []
#         prev_α = ''
#         for state_key, state in one_term.states.items():
#             (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
#             equiv_key = (Γ2, Γ1, Γ3, γ3, S, mSz)
#             if equiv_key in done_keys:
#                 continue
#                 warn_mark = '**'
#             else:
#                 warn_mark = ''
#             α = Γ1 * Γ2
#             multiplicity = 2*S+1
#             term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*S+1), ir = sp.latex(Γ3))
#             state_symbol = '%s \\Psi_{%d}(%s,%s,M\!=\!%d,%s)' % (warn_mark, counter+1, sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
#             state_symbol = sp.Symbol(state_symbol)
#             v_simple = Qet({(k[1],k[2],k[4],k[5]):v for k,v in state.dict.items()})
#             v_det = as_determinantal_ket(v_simple)
#             sup_key = (α)
#             if sup_key not in supreme_states.keys():
#                 supreme_states[sup_key] = []
#             supreme_states[sup_key].append(v_det)
#             pout = "$\\textcolor{blue}{%s} = %s$\\vspace{0.1cm}\n" % (sp.latex(state_symbol), sp.latex(v_det))
#             pout = sp.Symbol(pout)
#             if prev_α != α:
#                 if prev_α == '':
#                     print_outs.append('\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
#                 else:
#                     print_outs.append('\n\\vspace{0.25cm}\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
#             prev_α = α
#             print_outs.append(sp.latex(pout))
#             counter += 1
#             total_waves += 1
#             done_keys.append(state_key)
#         print_outs.append('\n\\end{center}\n\n')
#     irrep_combos = list(combinations_with_replacement(group.irrep_labels,2))
#     total_waves_groundtruth = sum([num_waves(ir0,ir1) for ir0, ir1 in irrep_combos])
#     tally_waves[group_label] = (total_waves,total_waves_groundtruth,full_waves())
# #     print_outs.append("Total--- waves = %d\n\n" % total_waves)
# #     print_outs.append("Expected waves = %d\n\n" % total_waves_groundtruth)
#     all_wavefunctions = '\n'.join(print_outs)
#     all_wavefunctions = all_wavefunctions.replace("^'","^{'}").replace("^''","^{''}")
#     all_final_outputs.append(all_wavefunctions)
# print("saving to file ...")
# super_final_output = '\n'.join(all_final_outputs)
# open('/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/termwaves.tex','w').write(super_final_output)
# !beep

Working on group T_{h} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
saving to file ...


In [ ]:
# def full_waves():
#     done = []
#     size = 0
#     for k,v in group.product_table.odict.items():
#         if (k[1],k[0]) in done:
#             continue
#         size += sum(list(map(lambda x: group.irrep_dims[x]*4, v)))
#         done.append(k)
#     return size

In [608]:
# irrep_combos = list(combinations_with_replacement(irreps,2))
# counter = 0
# for k,v in supreme_states.items():
#     α = k
#     combo = [(ir0,ir1) for ir0, ir1 in irrep_combos if ir0*ir1 == α][0]
#     display(α)
#     print("irrep dims",group.irrep_dims[combo[0]],group.irrep_dims[combo[1]])
#     if combo[0] == combo[1]:
#         print("nchoosek",sp.binomial(group.irrep_dims[combo[0]]*2,2))
#     else:
#         print("(d1*d2)*4",group.irrep_dims[combo[0]]*group.irrep_dims[combo[1]]*4)
#     print("waves---",len(v))
#     print("altsum--",sum(list(map(lambda x: group.irrep_dims[x],group.product_table.odict[combo])))*4)
#     counter += 1

In [609]:
# all_final_outputs = []
# for group_counter, group_label in enumerate(CPGs.all_group_labels):
#     if group_label != 'T_{h}':
#         continue
#     print("Working on group %s ..." % group_label)
#     group = CPGs.get_group_by_label(group_label)
#     print("computing partial sums...")
#     s1, s2 = sp.S(1)/2, sp.S(1)/2
#     Ss = [0,1]
#     m1s = [-sp.S(1)/2,sp.S(1)/2]
#     m2s = [-sp.S(1)/2,sp.S(1)/2]
#     group_CGs = group.CG_coefficients
#     summands = OrderedDict()
#     for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
#                                      group.irrep_labels,
#                                      group.irrep_labels,
#                                      m1s, m2s, Ss):
#         for γ1, γ2, γ3 in product(group.component_labels[Γ1],
#                                   group.component_labels[Γ2],
#                                   group.component_labels[Γ3]):
#             for mSz in range(-S, S+1):
#                 sCG = ClebschG(s1, s2, S, m1, m2, mSz)
#                 if (γ1, γ2, γ3) not in group_CGs.keys():
#                     gCG = 0
#                     continue
#                 else:
#                     gCG = group_CGs[(γ1, γ2, γ3)]
#                 coeff = sCG*gCG
#                 key = (Γ1, Γ2, Γ3, γ3, S, mSz)
#                 if (Γ2,m2,γ2) == (Γ1,m1,γ1):
#                     coeff = 0
#                 if coeff!=0:
#                     if key not in summands.keys():
#                         summands[key] = []
#                     summands[key].append(Qet({(Γ1,γ1,m1,Γ2,γ2,m2):coeff}))
#     print("summing out inner parts ...")
#     total_qets = OrderedDict([(k,sum(v,Qet({}))) for k,v in summands.items()])
#     best_qets = OrderedDict([(k,det_simplify(v)) for k,v in total_qets.items()])
#     best_qets = OrderedDict([(k,v) for k,v in best_qets.items() if len(v.dict)!=0])
#     print("term collection truck ...")
#     terms = OrderedDict()
#     for k, v in best_qets.items():
#         (Γ1, Γ2, Γ3, γ3, S, mSz) = k
#         term_pair = (Γ3, S)
#         if term_pair not in terms.keys():
#             terms[term_pair] = OrderedDict()
#         terms[term_pair][k] = v
#     final_terms = OrderedDict()
#     for term in terms:
#         ir, S = term
#         states = terms[term]
#         one_term = Term({'irrep': ir, 'S': S, 'states': states})
#         final_terms[term] = one_term
#     print("Manufacturing latex output ....")
#     max_counter = 3
# #     print_outs = ['\\begin{center} Group $%s$ \\end{center} \n' % group_label]
#     print_outs = ['\\subsection{Group $%s$} \n' % group_label]
#     for one_term_k, one_term in final_terms.items():
#         term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*one_term_k[1]+1), ir = sp.latex(one_term_k[0]))
#         term_symb = sp.Symbol(term_symb)
#         print_outs.append('\n\\hrulefill\n  \\subsubsection{$%s$} \n\\vspace{0.25cm}\n \\begin{center} \n' % sp.latex(term_symb))
#         counter = 0
#         done_keys = []
#         prev_α = ''
#         for state_key, state in one_term.states.items():
#             (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
#             equiv_key = (Γ2, Γ1, Γ3, γ3, S, mSz)
#             if equiv_key in done_keys:
#                 continue
#                 warn_mark = '**'
#             else:
#                 warn_mark = ''
#             α = Γ1 * Γ2
#             multiplicity = 2*S+1
#             term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*S+1), ir = sp.latex(Γ3))
#             state_symbol = '%s \\Psi_{%d}(%s,%s,M\!=\!%d,%s)' % (warn_mark, counter+1, sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
#             state_symbol = sp.Symbol(state_symbol)
#             v_simple = Qet({(k[1],k[2],k[4],k[5]):v for k,v in state.dict.items()})
#             v_det = as_determinantal_ket(v_simple)
#             pout = "$\\textcolor{blue}{%s} = %s$\\vspace{0.1cm}\n" % (sp.latex(state_symbol), sp.latex(v_det))
#             pout = sp.Symbol(pout)
#             if prev_α != α:
#                 if prev_α == '':
#                     print_outs.append('\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
#                 else:
#                     print_outs.append('\n\\vspace{0.25cm}\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
#             prev_α = α
#             print_outs.append(sp.latex(pout))
#             counter += 1
#             done_keys.append(state_key)
#         print_outs.append('\n\\end{center}')
#     all_wavefunctions = '\n'.join(print_outs)
#     all_wavefunctions = all_wavefunctions.replace("^'","^{'}").replace("^''","^{''}")
#     all_final_outputs.append(all_wavefunctions)
# #     if group_counter == 10:
# #         break
# print("saving to file ...")
# super_final_output = '\n'.join(all_final_outputs)
# open('/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/termwaves.tex','w').write(super_final_output)
# !beep

## $d^2$ (Oct 28)

In [32]:
class CrystalDefect():
    '''
    A CrystalDefect has the following attributes:
    
    - site          (str):
    - host          (str):
    - atom_symb     (str):
    - charge_state  (int):
    - ion           (Ion):
    - Bnm           (Bnm):
    - freeIparams  (dict):
    '''
    def __init__(self, defectDict):
        if 'Bnm' in defectDict.keys():
            self.Bnm = defectDict['Bnm']
            Bnmattrs = [a for a in dir(self.Bnm) if not a.startswith('__')]
            for Bnmattr in Bnmattrs:
                setattr(self, Bnmattr, getattr(self.Bnm, Bnmattr))
        for k, v in defectDict.items():
            if k == 'Bnm':
                continue
            setattr(self, k, v)
        self.GroupSymb = self.point_sym_group
        self.GroupNum = CPGs.all_group_labels.index(self.GroupSymb)
        self.Cfield = crystal_fields[self.GroupNum]
        self.splitter = crystal_splits[self.GroupNum]
    def __repr__(self):
        return self.Bnm.__repr__()

In [35]:
d2_df = CFparams[CFparams['nd^N'].apply(lambda x: 'd2' in x)]
Bnms = []
Cdefects = []
for row in d2_df.iterrows():
    row_data = row[1]
    Bparams = {k:v for k,v in row_data.items() if 'B_' in k}
    symmetry = row_data['Symmetry']
    if symmetry == 'C_{3i}':
        symmetry = 'S_{6}'
    Bnm_dict = {
        'host': row_data['Host'],
        'site': row_data['Site'],
        'point_sym_group': symmetry,
        'ion': row_data['Ion'],
        'params': Bparams,
        'unit' : 'cm^{-1}',
        'sources': row_data['Source'],
        'comments': '',
        'is_experimental': (row_data['Type'] == 'Experimental')
        }
    Bnms.append(Bnm(Bnm_dict))

### Term Factory

- Goal given an electron configuration, and a group, determine all of the terms that there could possibly exist.
- `term` contains
- 

In [115]:
class Term():
    '''
    Represents a term, holds the states that correspond to it,
    and offers a nice display of the term symbol.
    '''
    def __init__(self, init_dict):
        self.group_label = 'O_{h}'
        for k,v in init_dict.items():
            setattr(self, k, v)
        self.states = [] # a list of Qets
        self.term_prototype = sp.Symbol(r'\Psi({α},{S},{Γ},{M_s},{γ})')
    def term_symbol(self):
        ndict = {'α' : self.qnums[sp.Symbol(r'\alpha')],
        'S' : self.qnums[sp.Symbol('S')],
        'Γ' : self.qnums[sp.Symbol(r'\Gamma')],
        'M_s' : self.qnums[sp.Symbol(r'Ms')],
        'γ' : self.qnums[sp.Symbol(r'\gamma')]}
        return sp.Symbol(str(self.term_prototype).format(**ndict))
    def __str__(self):
        return str(self.qnums)
    def __repr__(self):
        return str(self.qnums)

In [116]:
aterm = Term({'qnums':{sp.Symbol(r'\alpha'):sp.Symbol(r'\alpha'), # qnum that distinguishes between states
                      sp.Symbol(r'S'):2, # The S in S(S+1), from eigenvalues of S^2
                      sp.Symbol(r'\Gamma'):sp.Symbol(r'T_{1}'), # symbol for irrep in group
                      sp.Symbol(r'\gamma'):sp.Symbol(r'\gamma'), # symbol for a component of a basis for the irrep
                      sp.Symbol(r'Ms'):1 # the Ms equal to m_1 + m_2
                      }
             })

In [117]:
group = CPGs.get_group_by_label('O')

In [126]:
group.product_table.odict[(sp.Symbol('T_1'),sp.Symbol('T_1'))]

[A_1, E, T_1, T_2]

In [118]:
group.component_labels

{A_1: [a_{A_1}],
 A_2: [a_{A_2}],
 E: [u_{E}, v_{E}],
 T_1: [x_{T_1}, y_{T_1}, z_{T_1}],
 T_2: [x_{T_2}, y_{T_2}, z_{T_2}]}

In [119]:
aterm.term_symbol()

\Psi(\alpha2T_{1}1\gamma)

In [120]:
aterm.term_prototype

\Psi({α},{S},{Γ},{M_s},{γ})

## $d^1$

In [3]:
crystal_splits[32]

{'matrices': [Matrix([
  [  B_{4,0}/21,             0,           0,             0, 5*B_{4,0}/21],
  [           0, -4*B_{4,0}/21,           0,             0,            0],
  [           0,             0, 2*B_{4,0}/7,             0,            0],
  [           0,             0,           0, -4*B_{4,0}/21,            0],
  [5*B_{4,0}/21,             0,           0,             0,   B_{4,0}/21]]),
  Matrix([
  [   B_{4,0}/21,             0,           0,             0, -5*B_{4,0}/21],
  [            0, -4*B_{4,0}/21,           0,             0,             0],
  [            0,             0, 2*B_{4,0}/7,             0,             0],
  [            0,             0,           0, -4*B_{4,0}/21,             0],
  [-5*B_{4,0}/21,             0,           0,             0,    B_{4,0}/21]])],
 'simple_free_parameters': [{B_{4,0}}, {B_{4,0}}],
 'free_parameters': [{B_{4,0}^r}, {B_{4,0}^r}],
 'eigen_system': [[(-4*B_{4,0}^r/21,
    3,
    [Matrix([
     [0],
     [1],
     [0],
     [0],
    

In [347]:
grup_indices = range(3,33)
matrices = {i: crystal_splits[i]['matrices'][0] for i in grup_indices}

In [349]:
grup_symbols = list(map(sp.Symbol,[CPGs.all_group_labels[i-1] for i in grup_indices]))

In [357]:
mahtrix = [grup_symbols]
umatrices = {}
for grup_index_0 in grup_indices:
    row = []
    m0 = matrices[grup_index_0]
    m0l = sp.latex(m0)
    if m0l not in umatrices.keys():
        umatrices[m0l] = []
    umatrices[m0l].append(CPGs.all_group_labels[grup_index_0-1])
    for grup_index_1 in grup_indices:
        m1 = matrices[grup_index_1]
        row.append(1 if m0==m1 else 0)
    mahtrix.append(row)
sp.Matrix(mahtrix)

Matrix([
[C_{2}, C_{s}, C_{2h}, D_{2}, C_{2v}, D_{2h}, C_{4}, S_{4}, C_{4h}, D_{4}, C_{4v}, D_{2d}, D_{4h}, C_{3}, S_{6}, D_{3}, C_{3v}, D_{3d}, C_{6}, C_{3h}, C_{6h}, D_{6}, C_{6v}, D_{3h}, D_{6h}, T, T_{h}, O, T_{d}, O_{h}],
[    1,     1,      1,     0,      0,      0,     0,     0,      0,     0,      0,      0,      0,     0,     0,     0,      0,      0,     0,      0,      0,     0,      0,      0,      0, 0,     0, 0,     0,     0],
[    1,     1,      1,     0,      0,      0,     0,     0,      0,     0,      0,      0,      0,     0,     0,     0,      0,      0,     0,      0,      0,     0,      0,      0,      0, 0,     0, 0,     0,     0],
[    1,     1,      1,     0,      0,      0,     0,     0,      0,     0,      0,      0,      0,     0,     0,     0,      0,      0,     0,      0,      0,     0,      0,      0,      0, 0,     0, 0,     0,     0],
[    0,     0,      0,     1,      1,      1,     0,     0,      0,     0,      0,      0,      0,     0,     0,     0,

In [12]:
class CrystalDefect():
    '''
    A CrystalDefect has the following attributes:
    
    - site          (str):
    - host          (str):
    - atom_symb     (str):
    - charge_state  (int):
    - ion           (Ion):
    - Bnm           (Bnm):
    - freeIparams  (dict):
    '''
    def __init__(self, defectDict):
        if 'Bnm' in defectDict.keys():
            self.Bnm = defectDict['Bnm']
            Bnmattrs = [a for a in dir(self.Bnm) if not a.startswith('__')]
            for Bnmattr in Bnmattrs:
                setattr(self, Bnmattr, getattr(self.Bnm, Bnmattr))
        for k, v in defectDict.items():
            if k == 'Bnm':
                continue
            setattr(self, k, v)
        self.GroupSymb = self.point_sym_group
        self.GroupNum = CPGs.all_group_labels.index(self.GroupSymb)
        self.Cfield = crystal_fields[self.GroupNum]
        self.splitter = crystal_splits[self.GroupNum]
    def __repr__(self):
        return self.Bnm.__repr__()

In [5]:
d1_df = CFparams[CFparams['nd^N'].apply(lambda x: 'd1' in x)]
Bnms = []
Cdefects = []
for row in d1_df.iterrows():
    row_data = row[1]
    Bparams = {k:v for k,v in row_data.items() if 'B_' in k}
    symmetry = row_data['Symmetry']
    if symmetry == 'C_{3i}':
        symmetry = 'S_{6}'
    Bnm_dict = {
        'host': row_data['Host'],
        'site': row_data['Site'],
        'point_sym_group': symmetry,
        'ion': row_data['Ion'],
        'params': Bparams,
        'unit' : 'cm^{-1}',
        'sources': row_data['Source'],
        'comments': '',
        'is_experimental': (row_data['Type'] == 'Experimental')
        }
    Bnms.append(Bnm(Bnm_dict))

In [9]:
CPGs.all_group_labels.index('S_{6}')

16

In [10]:
crystal_splits[16]['eigen_system'][0][1][0]

-B_{2,0}^r/14 - B_{4,0}^r/14 - sqrt(81*B_{2,0}^r**2 - 90*B_{2,0}^r*B_{4,0}^r + 25*B_{4,0}^r**2 + 140*B_{4,3}^i**2 + 140*B_{4,3}^r**2)/42

In [13]:
cdef = CrystalDefect({'Bnm':Bnms[0],'charge_state':4})

In [14]:
def square_field_in_a_round_hole(matrix, Bnm_params):
    # assume all params are real
    # and that they aint currently typed as either
    # assume that all that's missing is zero
    available_params = {sp.Symbol(k):v for k,v in Bnm_params.items() if not math.isnan(v)}
    matrix = matrix.subs(available_params)
    available_params = {sp.Symbol(k+'^r'):v for k,v in Bnm_params.items() if not math.isnan(v)}
    matrix_params = set(matrix.free_symbols)
    have_params = matrix_params.intersection(available_params)
    dont_have = {k: 0 for k in matrix_params.difference(available_params)}
    fudged_params = available_params
    fudged_params.update(dont_have)
    return matrix.subs(fudged_params)

In [15]:
cdef.splitter['matrices'][0].subs({sp.Symbol('B_{2,0}'):0,sp.Symbol('B_{4,3}^i'):0,sp.Symbol('B_{4,3}^r'):sp.sqrt(sp.S(10)/7) * sp.Symbol('B_{4,0}')}).eigenvals()

{2*B_{4,0}/7: 3, -3*B_{4,0}/7: 2}

In [16]:
splits = cdef.splitter['matrices'][0].subs({sp.Symbol('B_{2,0}'):0,sp.Symbol('B_{4,3}^i'):0,sp.Symbol('B_{4,3}^r'):sp.sqrt(sp.S(10)/7) * sp.Symbol('B_{4,0}')}).eigenvals()

In [18]:
cdef.host

'Y3Al5O12'

In [19]:
dE = (13200+8800.0) * UnitCon.con_factor('cm^{-1}','eV')
1240/dE

454.60338568141566

In [20]:
# splits = square_field_in_a_round_hole(cdef.splitter['matrices'][0], cdef.params).eigenvals()
splits = {-8800.0:3, 13200.0:2}
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4,6))
ax.plot([-1,0],[0,0])
for e, mult in splits.items():
    e = e*UnitCon.con_factor('cm^{-1}','eV') 
    for i in range(mult):
        ax.plot([0,1,2],[0,e+i/50,e+i/50],'k-')
ax.set_ylabel('$\Delta$E / $cm^{-1}$')
ax.set_title('$%s^{%d\!\!+}\!\!:\!%s$' % (cdef.ion, cdef.charge_state,''.join([r'{%s}_{%d}' % (k,v) for k,v in complex_formula_parser(cdef.host)[0].items()])))
ax.xaxis.set_visible(False)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
V4ion = Ion('V',4)
V4ion.nist_data.loc[:,'Level (cm^{-1})'] = V4ion.nist_data['Level (eV)'] * UnitCon.con_factor('eV','cm^{-1}')

In [29]:
V4ion.nist_data.loc[:,'Level (cm^{-1})']

0          0.000000
1        624.869951
2     148143.354049
3     206393.720573
4     207660.002906
          ...      
66    496296.308675
67    497556.146639
68    500116.956840
69    500501.683285
70    526532.016519
Name: Level (cm^{-1}), Length: 71, dtype: float64

In [30]:
cdef.splitter['free_parameters']

[{B_{2,0}^r, B_{4,0}^r, B_{4,3}^i, B_{4,3}^r}]

In [31]:
cdef.splitter['matrices'][0].free_symbols

{B_{2,0}, B_{4,0}, B_{4,3}^i, B_{4,3}^r}